In [7]:
import pandas
import os
import numpy as np

START_DATE = '2011-01-01'
END_DATE = '2017-01-01'

data = pandas.read_csv('features_SPY.csv', delimiter=',')
data['Date'] = pandas.to_datetime(data['Date'])

X = data.iloc[:, 0:23]
y = data['Date']
mask = (X['Date'] >= START_DATE) & (X['Date'] < END_DATE)

In [8]:
days = [0, 1, 2, 3, 5, 10, 20, 40, 60, 120, 250]

def getTarget(period):
    return_key = "Return_" + str(period)
    Y = data[return_key]
    maskTemp = mask
    maskTemp = maskTemp.shift(-period).fillna(False)
    Y = Y.shift(-period)
    Y = Y[maskTemp]
    Y = np.where(Y >= 0, 1.0, 0.0)
    #print(Y)
    #print(type(Y))
    return Y

#preparing features
def getFeaturesDataFrame(X, days, period):
    XfeaturesArray = __getFeatures(X, days, 2)
    XfeaturesMap = {"Feature" + str(i): XfeaturesArray[:,i] for i in range(0, XfeaturesArray[0].size)}
    return pandas.DataFrame(XfeaturesMap)

def __getFeatures(X, days, period):
    index_day = days.index(period)
    featureCount = (1 + days[index_day - 1]) * 2 + 2;
    
    
    #print("ii",index_day)
    return_key = "Return_" + str(period)
    volume_key = "AverageVolume_" + str(period)
    #print(return_key, volume_key)

    array = np.empty([0, featureCount])
    for index, row in X[mask].iterrows():
        #if (index > 2): break;
        features = np.empty(0)
        #print(row)
        column = X[return_key]
        #print(column[index])
        #print("i",index)
        #print("a", features)
        features = np.append(features, column[index])
        #print("aa", features)
        for lag in range(1, days[index_day - 1] + 1):
            #print("lag = %d" % lag)
            #numpy.append(features, value
            if (index - lag >= 0):
                features = np.append(features, column[index - lag])
            else:
                features = np.append(features, float("NaN"))
        
        column = X[volume_key]
        features = np.append(features, column[index])
        for lag in range(1, days[index_day - 1] + 1):
            #print("lag = %d" % lag)
            #numpy.append(features, value
            if (index - lag >= 0):
                features = np.append(features, column[index - lag])
            else:
                features = np.append(features, float("NaN"))

        column = X['DayOfWeek']
        features = np.append(features, column[index])
        column = X['DayOfMonth']
        features = np.append(features, column[index])
        
        #print(features)
        #https://stackoverflow.com/a/30305148
        #print(features[None, :].shape)
        #print(array.shape)
        array = np.vstack([array, features[None, :]])
        #print("array",array)
    #print('\n')
    
    return array

#print(__getFeatures(X, days, 250)[0])

In [9]:
from sklearn.model_selection import train_test_split

def getTrainDataset(period):
    XFeatures = getFeaturesDataFrame(X, days, period)
    XFeatures.reset_index(drop=True, inplace=True)
    X_train, X_test = train_test_split(XFeatures, test_size=0.3, random_state=241)
    #print(X_train.shape, XFeatures.shape)
    TrainFeatures = XFeatures.iloc[X_train.index, :]
    TrainFeatures.reset_index(drop=True, inplace=True)
    return TrainFeatures

def getTrainTarget(period):
    Target = getTarget(period)
    y_train, y_test = train_test_split(Target, test_size=0.3, random_state=241)
    #print(type(y_train),type(Target))
    #Target = Target[y_train]
    #Target = Target[y_train.index]
    #Target.reset_index(drop=True, inplace=True)
    return y_train

def getTestDataset(period):
    XFeatures = getFeaturesDataFrame(X, days, period)
    XFeatures.reset_index(drop=True, inplace=True)
    X_train, X_test = train_test_split(XFeatures, test_size=0.3, random_state=241)
    TestFeatures = XFeatures.iloc[X_test.index, :]
    TestFeatures.reset_index(drop=True, inplace=True)
    return TestFeatures

def getTestTarget(period):
    Target = getTarget(period)
    y_train, y_test = train_test_split(Target, test_size=0.3, random_state=241)
    #Target = Target[y_test]
    #Target = Target[y_test.index]
    #Target.reset_index(drop=True, inplace=True)
    return y_test

In [12]:
#TEST
#getTarget(1)
#(getTrainDataset(1).shape)
#getTrainTarget(1)
#getTestDataset(1)
#a = getTestTarget(1)


'getTarget(1)\n(getTrainDataset(1).shape)\ngetTrainTarget(1)\ngetTestDataset(1)\na = getTestTarget(1)\n'